<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import" data-toc-modified-id="Import-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import</a></span></li><li><span><a href="#1.-Measurement-분석-메타-생성-및-저장" data-toc-modified-id="1.-Measurement-분석-메타-생성-및-저장-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>1. Measurement 분석 메타 생성 및 저장</a></span><ul class="toc-item"><li><span><a href="#Setting-Upload-Param" data-toc-modified-id="Setting-Upload-Param-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Setting Upload Param</a></span></li><li><span><a href="#Get-Meta" data-toc-modified-id="Get-Meta-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Get Meta</a></span></li><li><span><a href="#Upload-Meta" data-toc-modified-id="Upload-Meta-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Upload Meta</a></span></li></ul></li><li><span><a href="#2.-DataBase-분석-메타-생성-및-저장" data-toc-modified-id="2.-DataBase-분석-메타-생성-및-저장-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>2. DataBase 분석 메타 생성 및 저장</a></span><ul class="toc-item"><li><span><a href="#Setting-Upload-Param" data-toc-modified-id="Setting-Upload-Param-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Setting Upload Param</a></span></li><li><span><a href="#Get-Meta" data-toc-modified-id="Get-Meta-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Get Meta</a></span></li><li><span><a href="#Upload-Meta" data-toc-modified-id="Upload-Meta-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Upload Meta</a></span></li></ul></li></ul></div>

# Import

In [2]:
import pandas as pd
import numpy as np
import json
from pprint import pprint

import sys
import os

sys.path.append("../")
sys.path.append("../..")

from Clust.setting import influx_setting_KETI as ins
from Clust.clust.ingestion.influx import influx_Client_v2 as iC

from Clust.clust.meta.metaDataManager import descriptor
from Clust.clust.meta.metaGenerator.analysisMSMetaGenerator import analysisMSMetaGenerator
from Clust.clust.meta.metaGenerator.analysisDBMetaGenerator import analysisDBMetaGenerator

from pprint import pprint

# 1. Measurement 분석 메타 생성 및 저장

## Setting Upload Param

In [3]:
#influx_instance = iC.influxClient(ins.CLUSTDataServer2)
mongo_instance = ins.wiz_url
print(mongo_instance)

http://10.252.107.59:5000


In [ ]:
## DataBase
domain = "farm"
sub_domain = "swine_air"

## 분석하고 싶은 measurement 설정
measurement_list = None

## 분석 함수 설정
function_list= None

In [ ]:
## 분석 메타 생성관련 Parameter
analysis_param = {
    "dbName":domain,
    "collectionName":sub_domain,
    "measurementList" : measurement_list,
    "functionList" : function_list
}

## Get Meta

In [ ]:
## 분석 메타 생성
meta_set = analysisMSMetaGenerator(analysis_param, influx_instance, mongo_instance).get_metaset()

In [ ]:
meta_set

In [ ]:
for m in meta_set:
    print(m["table_name"])
    print(m["analysisResult"]["StatisticsAnalyzer"].keys())
    print(m["analysisResult"]["MeanByHoliday"].keys())
    print(m["analysisResult"]["MeanByWorking"].keys())
    print(m["analysisResult"]["MeanByTimeStep"].keys())
    print(m["analysisResult"]["CountByFeatureLabel"].keys())
    print("------------------------------------------------------")

## Upload Meta

In [ ]:
## 생성된 메타 저장
# upload
uploadParam = {
    "dbName":domain,
    "collectionName":sub_domain,
    "mode" : "update"# insert / update / save
}

descriptor.write_data(uploadParam, meta_set, mongo_instance)

# 2. DataBase 분석 메타 생성 및 저장

## Setting Upload Param

In [ ]:
influx_instance = iC.influxClient(ins.CLUSTDataServer2)
mongo_instance = ins.wiz_url

In [ ]:
# 저장하고 싶은 Bucket 정보 셋팅
domain = "farm"
sub_domain = "swine_air"

In [ ]:
## 분석 메타 생성관련 Parameter
dbName = domain
collectionName = sub_domain
labels = {
        "StatisticsAnalyzer" : ["min", "max", "mean"],
        "MeanByHoliday" : ["holiday", "notHoliday"],
        "MeanByWorking" : ["working", "notWorking"],
        "MeanByTimeStep" : ["dawn", "morning", "afternoon", "evening", "night"]}

## Get Meta

In [ ]:
## 분석 메타 생성
gbam = analysisDBMetaGenerator(dbName, collectionName, labels, mongo_instance)
bucket_analysis_meta = gbam.get_bucketAnalysisMeta()

In [ ]:
bucket_analysis_meta

## Upload Meta

In [ ]:
## 생성된 메타 저장
uploadParam = {
    "dbName":"bucket",
    "collectionName":"meta_info",
    "mode" : "update"# insert / update / save
}

descriptor.write_data(uploadParam, bucket_analysis_meta, mongo_instance)
print("--------------------")